In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from mc_coda import estimate_ess
from mc_coda.effective_sample_size import _r_coda
from demo_helper import random_walk_metropolis_sampler, ProductGaussian

## Sample from a multi-variate Gaussian with random-walk Metropolis.

In [ ]:
dim = 100
target_sd = np.arange(1, dim + 1)
gaussian_target = ProductGaussian(marginal_sd=target_sd)
f = lambda x: gaussian_target.compute_logp(x)
proposal_sd = 2.38 / np.sqrt(dim) * target_sd
    # Optimal proposal sd: see Gelman, Roberts, and Gilks 1996.
    
n_samples = 10 ** 4
x0 = target_sd * np.random.randn(dim)
samples, accept_rate, logp_samples, _ = random_walk_metropolis_sampler(
    f, x0, proposal_sd, n_samples, seed=0, thin=5
)

In [ ]:
plt.figure(figsize=(13, 5))
plt.rcParams['font.size'] = 20

n_param_to_plot = 10
n_iter_to_plot = 2500
plt.plot(samples[:n_param_to_plot, :n_iter_to_plot].T)
plt.xlabel('MCMC iterations')
plt.title('Trace plot of the first {:d} parameters'.format(n_param_to_plot))
plt.tight_layout()

## Compare ESS computed by the module (via pure Python codes) with the one computed by R coda package.
_r_coda function calls the R coda package; it exists mainly for testing and is not meant for routine uses.

In [ ]:
ar_ess = estimate_ess(samples, axis=-1, method='ar')
r_coda_ess = _r_coda(samples, axis=-1)

In [ ]:
def plot_with_refline(x, y, min_val=None, max_val=None):
    
    if min_val is None:
        min_val = min(np.min(x), np.min(y))
    if max_val is None:
        max_val = max(np.max(x), np.max(y))
        
    plt.scatter(
        x, y, s=150, facecolors='none', edgecolors='C0'
    )
    plt.plot([min_val, max_val], [min_val, max_val], color='C1')
    plt.gca().set_aspect('equal', 'box')
    
    
plt.figure(figsize=(6, 6))
plt.rcParams['font.size'] = 20

plot_with_refline(ar_ess, r_coda_ess)
plt.xlabel('ESS by Python code')
plt.ylabel('ESS by R CODA')
plt.tight_layout()

## Other ESS estimators are availabe as well.
The initial monotone positive sequence estimator is valid *only* for reversible chains. The method of batch means tends to exhibit substantial bias and variances without a very long chain and a large number of batches.

In [ ]:
mono_seq_ess = estimate_ess(samples, axis=-1, method='monotone-sequence')
batch_ess = estimate_ess(
    samples, axis=-1, method='batch-means', options={'n_batch': 50}
)

In [ ]:
plt.figure(figsize=(13, 6.5))
plt.rcParams['font.size'] = 20

max_val = max(np.max(mono_seq_ess), np.max(batch_ess))

plt.subplot(1, 2, 1)
plot_with_refline(ar_ess, mono_seq_ess, 0, max_val)
plt.ylabel("ESS by monotone positive sequence")

plt.subplot(1, 2, 2)
plot_with_refline(ar_ess, batch_ess, 0, max_val)
plt.ylabel("ESS by method of batch means")

plt.gcf().text(.5, 0.02, 'ESS via AR model (as done in R CODA)', horizontalalignment='center')
plt.tight_layout()